In [1]:
pip install PyPDF2

     |████████████████████████████████| 81kB 4.0MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=52005b07029d2d0aa85f7e8191d73fc78cb1eedf88bf88ac7480c1e14b064445
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [380]:
import PyPDF2
import nltk
nltk.download('stopwords')
import operator
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import os
nltk.download('punkt')
import collections
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
data ={}

In [0]:
entries_1 =[]

In [0]:
entries = os.listdir('/content/drive/My Drive/PDF files ')

In [0]:
for f in range(len(entries)):
  entries_1.append("/content/drive/My Drive/PDF files /"+entries[f])

In [0]:
doc_data = {}
x =0

In [218]:
for f in entries_1:
  mypdf = open(f, mode='rb')
  pdf_document = PyPDF2.PdfFileReader(mypdf)
  first_page = pdf_document.getPage(0)
  page = first_page.extractText()
  tokens = word_tokenize(page)
  tokens = [w.lower() for w in tokens]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]
  words = [w for w in words if not w in alpha]
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  ans = " ".join(words)
  doc_data[entries[x]] = ans
  x = x+1
  if x%50 == 0:
    print(x)

50
100
150
200


In [0]:
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-91.pdf'] 
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-51.pdf'] 
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-253.pdf'] 

In [0]:

train_data =[]
for m,n in doc_data.items():
  file_string = nltk.word_tokenize(n)
  train_data.extend(file_string)
  #print(m)

In [0]:
vocabulary_size = 5000

def build_dataset(words):
  # Allocate a special token for rare words
  count = [['UNK', -1]]

  # Gets only the vocabulary_size most common words as the vocabulary
  # All the other words will be replaced with UNK token
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))


  # Create an ID for each word by giving the current length of the dictionary
  # And adding that item to the dictionary
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
    
  data = list()
  unk_count = 0
    
  # Traverse through all the text we have and produce a list
  # where each element corresponds to the ID of the word found at that index
  for word in words:
    # If word is in the dictionary use the word ID,
    # else use the ID of the special token "UNK"
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  
  # update the count variable with the number of UNK occurences
  count[0][1] = unk_count
    
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  # Make sure the dictionary is of size of the vocabulary
  assert len(dictionary) == vocabulary_size

  return data, count, dictionary, reverse_dictionary

def build_dataset_with_existing_dictionary(words, dictionary):
    '''
    Here we use this function to convert word strings to IDs
    with a given dictionary
    '''
    data = list()
    for word in words:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
        data.append(index)
    return data

# Processining training data
data, count, dictionary, reverse_dictionary = build_dataset(train_data)

In [223]:
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

Most common words (+UNK) [['UNK', 54], ('word', 334), ('example', 244), ('words', 212), ('data', 209)]
Sample data [6, 17, 569, 2567, 1382, 80, 6, 1100, 712, 14]


In [0]:
wordsunique = set(train_data)
wordsunique = list(wordsunique)

In [0]:
worddic={}

In [0]:
for doc_name,doc_content in doc_data.items():
  for word_1 in wordsunique:
    if word_1 in doc_content:
      index = doc_name
   #   position = list(np.where(np.array(doc_data[index]) == word_1))
    #  idfs = tfidf(word_1,doc_content,plottest)
      try:
        worddic[word_1].append([index])
      except:
        worddic[word_1] = []
        worddic[word_1].append([index])

In [0]:
np.save('worddic_10000.npy', worddic)

In [0]:
np.save("dictionary.npy",dictionary)

In [0]:
import numpy as np
df1 = pd.read_csv("/content/document_embeddings.csv")
document_embeddings1 = df1.to_dict('list')
word_embedding1 = np.load('/content/word_embedding.npy')

You can directly use above trained word and document embedding 

In [0]:
def build_dataset_with_existing_dictionary(words, dictionary):
    '''
    Here we use this function to convert word strings to IDs
    with a given dictionary
    '''
    words1 = words.split()
    data = list()
    for word in words1:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
        data.append(index)
    return data

In [0]:
test_data = {}
for k,v in doc_data.items():
  #  print('Building Test Dataset for ',k,' topic')
    test_data[k] = build_dataset_with_existing_dictionary(doc_data[k],dictionary)

In [231]:
data_index = 0

def generate_batch_cbow(batch_size, window_size):
    # window_size is the amount of words we're looking at from each side of a given word
    # creates a single batch
    
    # data_index is updated by 1 everytime we read a set of data point
    global data_index

    # span defines the total window size, where
    # data we consider at an instance looks as follows. 
    # [ skip_window target skip_window ]
    # e.g if skip_window = 2 then span = 5
    span = 2 * window_size + 1 # [ skip_window target skip_window ]

    # two numpy arras to hold target words (batch)
    # and context words (labels)
    # Note that batch has span-1=2*window_size columns
    batch = np.ndarray(shape=(batch_size,span-1), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # The buffer holds the data contained within the span
    buffer = collections.deque(maxlen=span)

    # Fill the buffer and update the data_index
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    # Here we do the batch reading
    # We iterate through each batch index
    # For each batch index, we iterate through span elements
    # to fill in the columns of batch array
    for i in range(batch_size):
        target = window_size  # target label at the center of the buffer
        target_to_avoid = [ window_size ] # we only need to know the words around a given word, not the word itself

        # add selected target to avoid_list for next time
        col_idx = 0
        for j in range(span):
            # ignore the target word when creating the batch
            if j==span//2:
                continue
            batch[i,col_idx] = buffer[j] 
            col_idx += 1
        labels[i, 0] = buffer[target]

        # Everytime we read a data point,
        # we need to move the span by 1
        # to create a fresh new span
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    return batch, labels

for window_size in [1,2]:
    data_index = 0
    batch, labels = generate_batch_cbow(batch_size=8, window_size=window_size)
    print('\nwith window_size = %d:' % (window_size))
    print('    batch:', [[reverse_dictionary[bii] for bii in bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])



with window_size = 1:
    batch: [['lstm', 'seen'], ['lstms', 'fancier'], ['seen', 'family'], ['fancier', 'rnns'], ['family', 'lstm'], ['rnns', 'composed'], ['lstm', 'mainly'], ['composed', 'cell']]
    labels: ['lstms', 'seen', 'fancier', 'family', 'rnns', 'lstm', 'composed', 'mainly']

with window_size = 2:
    batch: [['lstm', 'lstms', 'fancier', 'family'], ['lstms', 'seen', 'family', 'rnns'], ['seen', 'fancier', 'rnns', 'lstm'], ['fancier', 'family', 'lstm', 'composed'], ['family', 'rnns', 'composed', 'mainly'], ['rnns', 'lstm', 'mainly', 'cell'], ['lstm', 'composed', 'cell', 'state'], ['composed', 'mainly', 'state', 'internal']]
    labels: ['seen', 'fancier', 'family', 'rnns', 'lstm', 'composed', 'mainly', 'cell']


In [232]:
test_data_index = 0

def generate_test_batch(data, batch_size1):
    '''
    Generate a batch of data from the test data
    This is used to compute the document embedding
    by taking the average of all the words in a document
    '''
    global test_data_index

    batch = np.ndarray(shape=(batch_size1,), dtype=np.int32)
    # Get words starting from index 0 to span
    for bi in range(batch_size1):
        batch[bi] = data[test_data_index]
        test_data_index = (test_data_index + 1) % len(data)

    return batch

test_data_index = 0
test_batch = generate_test_batch(test_data["Natural Language Processing with TensorFlow ( PDFDrive.com )-248.pdf"], 128)
print('\nwith window_size = %d:' % (window_size))
print('    labels:', [reverse_dictionary[li] for li in test_batch.reshape(128)])


with window_size = 2:
    labels: ['peephole', 'connectionspeephole', 'connections', 'allow', 'gates', 'see', 'current', 'input', 'previous', 'weights', 'lstm', 'cell', 'connections', 'shown', 'produce', 'better', 'results', 'equations', 'would', 'look', 'like', 'ihti', 'ct', 'iiw', 'xw', 'hw', 'cb', 'chtc', 'ct', 'anhwx', 'wh', 'fhtf', 'ct', 'ffw', 'xw', 'hw', 'cb', 'tt', 'tcf', 'ci', 'ohto', 'ct', 'oow', 'xw', 'hw', 'cb', 'tt', 'tho', 'tanhclook', 'helps', 'lstm', 'perform', 'better', 'far', 'gates', 'see', 'close', 'zero', 'thus', 'gates', 'take', 'hidden', 'state', 'consideration', 'calculation', 'including', 'cell', 'state', 'directly', 'gate', 'calculation', 'equation', 'allows', 'control', 'cell', 'state', 'perform', 'well', 'even', 'situations', 'output', 'gate', 'close', 'zero', 'peephole', 'connectionspeephole', 'connections', 'allow', 'gates', 'see', 'current', 'input', 'previous', 'weights', 'lstm', 'cell', 'connections', 'shown', 'produce', 'better', 'results', 'equations

In [0]:
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
batch_size = 128 # Data points in a single batch
embedding_size = 32 # Dimension of the embedding vector.
# How many words to consider left and right.
# Skip gram by design does not require to have all the context words in a given step
# However, for CBOW that's a requirement, so we limit the window size
window_size = 4

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 50

# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words as well
valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.append(valid_examples,random.sample(range(1000, 1000+valid_window), valid_size),axis=0)

num_sampled = 32 # Number of negative examples to sample.

In [0]:
tf.reset_default_graph()

# Training input data (target word IDs). Note that it has 2*window_size columns
train_dataset = tf.placeholder(tf.int32, shape=[batch_size,2*window_size])
# Training input label data (context word IDs)
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
# Validation input data, we don't need a placeholder
# as we have already defined the IDs of the words selected
# as validation data
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [0]:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0,dtype=tf.float32))

# Softmax Weights and Biases
softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                 stddev=0.5 / math.sqrt(embedding_size),dtype=tf.float32))
softmax_biases = tf.Variable(tf.random_uniform([vocabulary_size],0.0,0.01))

In [271]:
stacked_embedings = None
print('Defining %d embedding lookups representing each word in the context'%(2*window_size))
for i in range(2*window_size):
    embedding_i = tf.nn.embedding_lookup(embeddings, train_dataset[:,i])        
    x_size,y_size = embedding_i.get_shape().as_list()
    if stacked_embedings is None:
        stacked_embedings = tf.reshape(embedding_i,[x_size,y_size,1])
    else:
        stacked_embedings = tf.concat(axis=2,values=[stacked_embedings,tf.reshape(embedding_i,[x_size,y_size,1])])

assert stacked_embedings.get_shape().as_list()[2]==2*window_size
print("Stacked embedding size: %s"%stacked_embedings.get_shape().as_list())
mean_embeddings =  tf.reduce_mean(stacked_embedings,2,keepdims=False)
print("Reduced mean embedding size: %s"%mean_embeddings.get_shape().as_list())

# Compute the softmax loss, using a sample of the negative labels each time.
# inputs are embeddings of the train words
# with this loss we optimize weights, biases, embeddings
loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=mean_embeddings,
                           labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

Defining 8 embedding lookups representing each word in the context
Stacked embedding size: [128, 32, 8]
Reduced mean embedding size: [128, 32]


In [0]:
optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)


In [0]:
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [0]:
test_labels = tf.placeholder(tf.int32, shape=[batch_size],name='test_dataset')

mean_batch_embedding = tf.reduce_mean(tf.nn.embedding_lookup(embeddings,test_labels),axis=0)

In [275]:
num_steps = 200001
cbow_loss = []

config=tf.ConfigProto(allow_soft_placement=True)
# This is an important setting and with limited GPU memory,
# not using this option might lead to the following error.
# InternalError (see above for traceback): Blas GEMM launch failed : ...
config.gpu_options.allow_growth = True

with tf.Session(config=config) as session:
    
    # Initialize the variables in the graph
    tf.global_variables_initializer().run()
    print('Initialized')
    
    average_loss = 0
    
    # Train the Word2vec model for num_step iterations
    for step in range(num_steps):
        
        # Generate a single batch of data
        batch_data, batch_labels = generate_batch(data, batch_size, window_size)
        
        # Populate the feed_dict and run the optimizer (minimize loss)
        # and compute the loss
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        
        # Update the average loss variable
        average_loss += l
        
        if (step+1) % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step+1, average_loss))
            cbow_loss.append(average_loss)
            average_loss = 0
        
        # Evaluating validation set word similarities
       
    
    # Computing test documents embeddings by averaging word embeddings
    
    # We take batch_size*num_test_steps words from each document
    # to compute document embeddings
    num_test_steps = 100
    
    # Store document embeddings
    # {document_id:embedding} format
    document_embeddings = {}
    print('Testing Phase (Compute document embeddings)')
  
      # For each test document compute document embeddings
    for k,v in test_data.items():
        print('\tCalculating mean embedding for document ',k,' with ', num_test_steps, ' steps.')
        test_data_index = 0
        topic_mean_batch_embeddings = np.empty((num_test_steps,embedding_size),dtype=np.float32)
        
        # keep averaging mean word embeddings obtained for each step
        for test_step in range(num_test_steps):
            test_batch_labels = generate_test_batch(test_data[k],batch_size)
            batch_mean = session.run(mean_batch_embedding,feed_dict={test_labels:test_batch_labels})
            topic_mean_batch_embeddings[test_step,:] = batch_mean
        document_embeddings[k] = np.mean(topic_mean_batch_embeddings,axis=0)
        
    word_embedding = session.run(embeddings)

Initialized
Average loss at step 2000: 3.586534
Average loss at step 4000: 3.181315
Average loss at step 6000: 2.987666
Average loss at step 8000: 2.827531
Average loss at step 10000: 2.713530
Average loss at step 12000: 2.636535
Average loss at step 14000: 2.530674
Average loss at step 16000: 2.443517
Average loss at step 18000: 2.392001
Average loss at step 20000: 2.332445
Average loss at step 22000: 2.290571
Average loss at step 24000: 2.256202
Average loss at step 26000: 2.230088
Average loss at step 28000: 2.164433
Average loss at step 30000: 2.147199
Average loss at step 32000: 2.143273
Average loss at step 34000: 2.115013
Average loss at step 36000: 2.088984
Average loss at step 38000: 2.072071
Average loss at step 40000: 2.064901
Average loss at step 42000: 2.022371
Average loss at step 44000: 2.026131
Average loss at step 46000: 2.018900
Average loss at step 48000: 1.996160
Average loss at step 50000: 2.002527
Average loss at step 52000: 1.984686
Average loss at step 54000: 1.

In [0]:
import pandas as pd
df = pd.DataFrame.from_dict(document_embeddings)

In [0]:
df.to_csv("document_embeddings.csv")

In [0]:
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]


In [0]:
query = "gradient clipping"
quer_list = []
def filter_query(q):
  tokens = q.split()
  tokens = [w.lower() for w in tokens]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]
  words = [w for w in words if not w in alpha]
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  return words

In [0]:
query_list= filter_query(query)

In [0]:
related_docs=[]

for f in query_list:
  a = worddic[f]
  for x in a:
    related_docs.append(x[0])

In [0]:
query_embedding =[]
for f in query_list:
  query_embedding.append(word_embedding1[dictionary[f]])

query_embedding_mean = np.mean(np.array(query_embedding),axis=0)

In [0]:
related_docs_unique = list(set(related_docs))

In [0]:
def similar_func(related_docs_unique,query_embedding_mean):                               

  similarity={}

  for f in range(len(related_docs_unique)):
    score= get_cosine_similarity(document_embeddings[related_docs_unique[f]],query_embedding_mean)
    similarity[related_docs_unique[f]]=score

  similarity_sort=sorted(similarity.items(), key=operator.itemgetter(1),reverse=True)
  return(similarity_sort)

In [436]:
similar_func(related_docs_unique,query_embedding_mean)

[('Natural Language Processing with TensorFlow ( PDFDrive.com )-240.pdf',
  0.6701666),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-263.pdf',
  0.58353657),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-241.pdf',
  0.5826148),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-239.pdf',
  0.53367114),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-289.pdf',
  0.5116844),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-238.pdf',
  0.49216157),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-225.pdf',
  0.38376698),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-226.pdf',
  0.34090504),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-44.pdf',
  0.33073175),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-227.pdf',
  0.31529427),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-252.pdf',
  0.25295624),
 ('Natural Language Proce